In [2]:
import numpy as np
from collections import Counter

# ----------------------------
# Decision Tree (with Gini Index)
# ----------------------------
class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value  # Only for leaf nodes


class DecisionTree:
    def __init__(self, min_samples_split=2, max_depth=100, n_features=None):
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_features = n_features
        self.root = None

    # ---------- Public fit ----------
    def fit(self, X, y):
        self.n_features = X.shape[1] if not self.n_features else min(self.n_features, X.shape[1])
        self.root = self._grow_tree(X, y)

    # ---------- Recursive tree growth ----------
    def _grow_tree(self, X, y, depth=0):
        n_samples, n_features = X.shape
        n_labels = len(np.unique(y))

        # stopping conditions
        if (depth >= self.max_depth or 
            n_labels == 1 or 
            n_samples < self.min_samples_split):
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        feat_idxs = np.random.choice(n_features, self.n_features, replace=False)

        # find best split
        best_feat, best_thresh = self._best_split(X, y, feat_idxs)

        # if no valid split found → make leaf
        if best_feat is None:
            leaf_value = self._most_common_label(y)
            return Node(value=leaf_value)

        # split data
        left_idxs = np.argwhere(X[:, best_feat] <= best_thresh).flatten()
        right_idxs = np.argwhere(X[:, best_feat] > best_thresh).flatten()

        # recursive step
        left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)
        right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)
        return Node(best_feat, best_thresh, left, right)

    # ---------- Best split ----------
    def _best_split(self, X, y, feat_idxs):
        best_gain = -1
        split_idx, split_thresh = None, None

        for feat_idx in feat_idxs:
            thresholds = np.unique(X[:, feat_idx])
            for thresh in thresholds:
                left_idxs = np.argwhere(X[:, feat_idx] <= thresh).flatten()
                right_idxs = np.argwhere(X[:, feat_idx] > thresh).flatten()

                # 🚨 skip if split is invalid
                if len(left_idxs) == 0 or len(right_idxs) == 0:
                    continue

                gain = self._information_gain(y, left_idxs, right_idxs)

                if gain > best_gain:
                    best_gain = gain
                    split_idx = feat_idx
                    split_thresh = thresh

        return split_idx, split_thresh

    # ---------- Gini Impurity ----------
    def _gini(self, y):
        m = len(y)
        counts = np.bincount(y)
        impurity = 1.0 - sum((count / m) ** 2 for count in counts if count > 0)
        return impurity

    # ---------- Information Gain ----------
    def _information_gain(self, y, left_idxs, right_idxs):
        # parent loss
        parent_loss = self._gini(y)

        # weighted avg. child loss
        n = len(y)
        n_l, n_r = len(left_idxs), len(right_idxs)
        if n_l == 0 or n_r == 0:
            return 0

        e_l, e_r = self._gini(y[left_idxs]), self._gini(y[right_idxs])
        child_loss = (n_l / n) * e_l + (n_r / n) * e_r

        # gain
        return parent_loss - child_loss

    # ---------- Helper ----------
    def _most_common_label(self, y):
        counter = Counter(y)
        return counter.most_common(1)[0][0]

    # ---------- Prediction ----------
    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X])

    def _traverse_tree(self, x, node):
        if node.value is not None:
            return node.value
        if x[node.feature] <= node.threshold:
            return self._traverse_tree(x, node.left)
        return self._traverse_tree(x, node.right)


# ----------------------------
# Random Forest
# ----------------------------
class RandomForest:
    def __init__(self, n_trees=10, min_samples_split=2, max_depth=100, n_features=None):
        self.n_trees = n_trees
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.n_features = n_features
        self.trees = []

    def fit(self, X, y):
        self.trees = []
        for _ in range(self.n_trees):
            tree = DecisionTree(
                min_samples_split=self.min_samples_split,
                max_depth=self.max_depth,
                n_features=self.n_features
            )
            # bootstrap sample
            idxs = np.random.choice(len(X), len(X), replace=True)
            X_sample, y_sample = X[idxs], y[idxs]
            tree.fit(X_sample, y_sample)
            self.trees.append(tree)

    def predict(self, X):
        # collect predictions from each tree
        tree_preds = np.array([tree.predict(X) for tree in self.trees])
        # majority vote across trees
        tree_preds = np.swapaxes(tree_preds, 0, 1)  # shape: (n_samples, n_trees)
        y_pred = [Counter(tree_pred).most_common(1)[0][0] for tree_pred in tree_preds]
        return np.array(y_pred)


# ----------------------------
# Example Run
# ----------------------------
if __name__ == "__main__":
    from sklearn.datasets import load_iris
    from sklearn.model_selection import train_test_split

    data = load_iris()
    X, y = data.data, data.target
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    clf = RandomForest(n_trees=5, max_depth=10)
    clf.fit(X_train, y_train)
    preds = clf.predict(X_test)

    acc = np.sum(preds == y_test) / len(y_test)
    print("Accuracy:", acc)


Accuracy: 1.0
